In [1]:
from PIL import Image
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from skimage.transform import resize
from os.path import join

import tensorflow as tf
import matplotlib.pyplot as plt
import torch

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
from matplotlib import image as mpimg 

# Your data directory
data_dir = r"E:\Documentos\01_DTU\DTU-TFM\Fotos\Real DOE\UV\LDPE black"
image_exts = ['.tif']

# Load the model
model = tf.keras.models.load_model('CNN.model')

# Assuming df is your DataFrame containing 'Image' and 'Contrast' columns
# (Replace this with your actual DataFrame)
df = pd.read_csv(r"E:\Documentos\02_Programming\A-data-driven-approach-to-marking-of-plastic-materials-by-laser-micro-machining\Image processing\Contrast and Histograms\Real DOE\Contrast_results_DOE__UV_PP_Black.csv")
label_dict = pd.Series(df['Contrast'].values, index=df['Image']).to_dict()

# Get a list of all .tif files in the directory
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if any(f.endswith(ext) for ext in image_exts)]

# Choose only the first 15 images for testing
image_files_subset = image_files[:15]

# Load Images and Labels
images = []
labels = []

# New size for the resized images
new_size = (512, 512)

for image_file in tqdm(image_files_subset, desc="Loading Images"):

    # Load image
    img = Image.open(image_file)
    img = img.resize(new_size)  # Resize the image
    img = np.array(img) / 255.0  # Convert to numpy array and normalize pixel values
    img = tf.cast(img, tf.float32) / 255.0
    
    # Get the corresponding label from the dictionary
    label = label_dict.get(os.path.basename(image_file), 'unknown')  # Use 'unknown' if label is not in the dictionary
    
    # Append to lists
    images.append(img)
    labels.append(label)

# Convert lists to arrays
images = np.array(images)
labels = np.array(labels)

# Check the shape of the first resized image and label
print("Shape of the first resized image:", images[0].shape)
print("Label of the first image:", labels[0])

# Predict using the loaded model
predictions = model.predict(images)

# Visualize the images with actual labels and predicted contrasts
for i in range(len(images)):
    plt.figure(figsize=(8, 8))
    
    # Actual image with actual label
    plt.subplot(2, 1, 1)
    img_actual = mpimg.imread(image_files_subset[i])
    plt.imshow(img_actual)
    plt.title(f"Actual Label: {labels[i]}")
    plt.axis('off')
    
    # Predicted contrast below the actual image
    plt.subplot(2, 1, 2)
    plt.plot([-100, 100], [-100, 100], '--', color='gray')  # Diagonal line
    plt.scatter(labels[i], predictions[i], color='red')
    plt.title(f"Predicted Contrast: {predictions[i][0]:.2f}")
    plt.xlabel('Actual Contrast')
    plt.ylabel('Predicted Contrast')
    
    plt.show()

Loading Images: 100%|██████████| 15/15 [00:02<00:00,  7.10it/s]


Shape of the first resized image: (512, 512, 3)
Label of the first image: unknown


ValueError: in user code:

    File "c:\Anaconda\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Anaconda\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Anaconda\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Anaconda\envs\tf2.5\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Anaconda\envs\tf2.5\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Anaconda\envs\tf2.5\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of laye